In [215]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal
import requests
dec = decimal.Decimal
import hvplot.pandas, holoviews as hv
import json

def get_crypto_price(symbol, exchange, days):
    api_key = 'YOUR API KEY'
    api_url = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym={exchange}&limit={days}&api_key={api_key}'
    raw = requests.get(api_url).json()
    df = pd.DataFrame(raw['Data']['Data'])[['time', 'close']].set_index('time')
    df.index = pd.to_datetime(df.index, unit = 's')
    return df
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json()

In [216]:
from pathlib import Path
csvpath1 = Path("./full_moon.csv")
lunar_eclipse = Path("./lunar_eclipse.csv")
solar_eclipse = Path("./solar_eclipse.csv")
mercury_retro = Path("./mercury_retrograde.csv")

# Retrieve BTC API & Create Trading Signals

In [217]:
# BTC API data
btc = get_crypto_price('BTC', 'USD', 1825)
Price = btc['Price'] = btc['close']

# create signals and store in a dataframe
SMA_9 = btc['9SMA'] = btc['close'].rolling(9).mean()
SMA_44 = btc['44SMA'] = btc['close'].rolling(44).mean()
SMA_117 = btc['117SMA'] = btc['close'].rolling(117).mean()
EMA_117 = btc['117EMA'] = btc['close'].ewm(117).mean()
btc = btc.reset_index()
btc = btc.rename(columns={'time':'dtime'})
btc.set_index(['dtime'], inplace=True)
btc_api = btc.drop(['close'], axis=1)
btc_api

,Price,9SMA,44SMA,117SMA,117EMA
dtime,,,,,
2017-06-30,2480.61,NaN,NaN,NaN,2480.610000
2017-07-01,2424.61,NaN,NaN,NaN,2452.490851
2017-07-02,2536.46,NaN,NaN,NaN,2480.719114
2017-07-03,2572.47,NaN,NaN,NaN,2503.950485
2017-07-04,2617.32,NaN,NaN,NaN,2527.011962
...,...,...,...,...,...
2022-06-25,21474.19,20548.703333,27170.538182,35507.560940,37972.971726
2022-06-26,21031.85,20615.324444,26983.879545,35307.589829,37829.402871
2022-06-27,20718.16,20811.314444,26771.785227,35109.152137,37684.392312


### Plot BTC Price over Time

In [218]:
BTC = btc_api.hvplot.line(
    x = 'dtime',
    y = 'Price',
    xlabel = 'Date',
    ylabel = 'Price',
    title = 'BTC Price',
    legend = True
)
BTC

:Curve   [dtime]   (Price)

# Clean Lunar Data


In [219]:
#Create a Full Moon Dataframe
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'Full_Moon'})
full_moon.drop([' Time'], axis=1, inplace=True)

In [220]:
# Create a Moon Data table with the BTC Price data for each Full and New Moon date

full_moon = full_moon.join(btc['Price'], on = 'Full_Moon', how = 'inner')
full_moon['Full Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price'])

full_moon.tail()

,Full_Moon,Full Moon Price
53,2022-02-16,43896.34
54,2022-03-18,41790.97
55,2022-04-16,40392.55
56,2022-05-16,29838.50
57,2022-06-14,22118.37


In [221]:
#Create a New Moon df
new_moon = pd.DataFrame(full_moon['Full_Moon'] + timedelta(days=15))
new_moon.rename(columns={'Full_Moon':'New_Moon'}, inplace=True)
new_moon.drop(new_moon.index[-1], inplace=True)
new_moon = new_moon.rename(columns = {'Full_Moon':'New_Moon'})
new_moon = new_moon.join(btc['Price'], on = 'New_Moon', how = 'inner')
new_moon['New Moon Price'] = new_moon['Price']
new_moon = new_moon.drop(columns=['Price'])

new_moon.tail()

,New_Moon,New Moon Price
52,2022-02-02,36918.19
53,2022-03-03,42474.37
54,2022-04-02,45822.28
55,2022-05-01,38480.53
56,2022-05-31,31782.16


In [222]:
moon_data = pd.concat([new_moon,full_moon], axis=1, join='inner')
moon_data

,New_Moon,New Moon Price,Full_Moon,Full Moon Price
0,2017-11-19,8042.64,2017-11-04,7363.80
1,2017-12-18,18972.32,2017-12-03,11246.21
2,2018-01-17,11162.70,2018-01-02,14754.13
3,2018-02-15,10033.75,2018-01-31,10226.86
4,2018-03-17,7883.45,2018-03-02,11043.12
5,2018-04-15,8376.73,2018-03-31,6943.77
6,2018-05-15,8480.16,2018-04-30,9248.45
7,2018-06-13,6310.43,2018-05-29,7474.75
8,2018-07-13,6229.83,2018-06-28,5871.28
9,2018-08-11,6242.54,2018-07-27,8182.89


In [223]:
lunar_eclipse_df = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True)
lunar_eclipse_df.rename(columns={'Date':'lunar eclipse'}, inplace=True)
lunar_eclipse_df['lunar eclipse'] = pd.to_datetime(lunar_eclipse_df['lunar eclipse'])

lunar_eclipse_df = lunar_eclipse_df.join(btc['Price'], on = 'lunar eclipse', how = 'inner')

In [224]:
solar_eclipse_df = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True)
solar_eclipse_df.rename(columns={'Date':'solar eclipse'}, inplace=True)
solar_eclipse_df['solar eclipse'] = pd.to_datetime(solar_eclipse_df['solar eclipse'])
solar_eclipse_df.drop(['Unnamed: 1'], axis = 1, inplace=True)

solar_eclipse_df = solar_eclipse_df.join(btc['Price'], on = 'solar eclipse', how = 'inner')

In [225]:
mercury_data = pd.read_csv(mercury_retro, parse_dates=True, infer_datetime_format=True)
mercury_data['Start'] = pd.to_datetime(mercury_data['Start'])
mercury_data['End'] = pd.to_datetime(mercury_data['End'])
mercury_data['Term'] = mercury_data['End'] - mercury_data['Start']
# mercury_data = pd.date_range(start=mercury_data['Start'], end=mercury_data['End'])
start = mercury_data['Start']
start_hv = start.hvplot.scatter()
end = mercury_data['End']
end_hv = end.hvplot.scatter()

mercury_data

# start_hv*end_hv

,Start,End,Term
0,2017-12-03,2017-12-22,19 days
1,2018-03-22,2018-04-15,24 days
2,2018-07-26,2018-08-18,23 days
3,2018-11-16,2018-12-06,20 days
4,2019-03-05,2019-03-28,23 days
5,2019-07-07,2019-07-31,24 days
6,2019-10-31,2019-11-20,20 days
7,2020-02-16,2020-03-09,22 days
8,2020-06-17,2020-07-12,25 days
9,2020-10-13,2020-11-03,21 days


# Plot Lunar Data over BTC Data

In [226]:
# create hvplot figures to then overaly 
glyph_1 = btc['Price'].hvplot.line( 
    'dtime', 'Price',
    color='#e7e7e7',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_2 = full_moon.hvplot.scatter(
    x = 'Full_Moon',
    y = 'Full Moon Price',
    color='#ffcd33',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_3 = new_moon.hvplot.scatter(
    x = 'New_Moon',
    y = 'New Moon Price',
    color='#ff6533',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_4 = btc['9SMA'].hvplot.line(
    'dtime', '9SMA',
    color='#70eac4',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_9 = btc['44SMA'].hvplot.line(
    'dtime', '44SMA',
    color='pink',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_5 = btc['117SMA'].hvplot.line(
    'dtime', '117SMA',
    color='#55d24a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_6 = btc['117EMA'].hvplot.line(
    'dtime', '117EMA',
    color='#22a91a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_7 = lunar_eclipse_df.hvplot.scatter(
    x = 'lunar eclipse',
    y = 'Price',
    color='#505050',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_8 = solar_eclipse_df.hvplot.scatter(
    x = 'solar eclipse',
    y = 'Price',
    color='#000000',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_1*glyph_2*glyph_3*glyph_4*glyph_5*glyph_6*glyph_9*glyph_7*glyph_8

:Overlay
   .Curve.Price    :Curve   [dtime]   (Price)
   .Scatter.I      :Scatter   [Full_Moon]   (Full Moon Price)
   .Scatter.II     :Scatter   [New_Moon]   (New Moon Price)
   .Curve.A_9SMA   :Curve   [dtime]   (9SMA)
   .Curve.A_117SMA :Curve   [dtime]   (117SMA)
   .Curve.A_117EMA :Curve   [dtime]   (117EMA)
   .Curve.A_44SMA  :Curve   [dtime]   (44SMA)
   .Scatter.III    :Scatter   [lunar eclipse]   (Price)
   .Scatter.IV     :Scatter   [solar eclipse]   (Price)

If New Moon is < Previous Full Moon = Do not sell.
Do not buy on full moon before eclipse.
When the 8 SMA > 200 SMA > 200 EMA = Bearish Cross, do not buy on full moon.
When the 8 SMA < 200 SMA > 200 EMA = Buy the full moon

## Prepare Lunar and BTC Data for Merge

In [227]:
btc = get_crypto_price('BTC', 'USD', 1825)
Price = btc['Price'] = btc['close']
SMA_8 = btc['8SMA'] = btc['close'].rolling(8).mean()
SMA_200 = btc['200SMA'] = btc['close'].rolling(200).mean()
EMA_200 = btc['200EMA'] = btc['close'].ewm(200).mean()
btc = btc.reset_index()
btc = btc.rename(columns={'time':'dtime'})
# btc.set_index(['dtime'], inplace=True)
btc_api = btc.drop(['close'], axis=1)
# btc_api

In [228]:
# Read Full_Moon CSV, Clean index, infer Datetime
full_moon = pd.read_csv(csvpath1)
full_moon = full_moon.rename(columns = {' Date':'dtime'})
full_moon['dtime'] = full_moon['dtime']+' '+full_moon[' Time']
full_moon['Phase'] = full_moon.loc['dtime',:] = 1
full_moon.drop([' Time'], axis=1, inplace=True)
full_moon.drop(full_moon.index[-1], inplace=True)

full_moon['dtime'] = pd.to_datetime(full_moon['dtime'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
full_moon['dtime'] = pd.DataFrame(full_moon['dtime'] + timedelta(hours=-2))

full_moon

,dtime,Phase
0,2017-11-04 04:22:55,1.0
1,2017-12-03 14:46:59,1.0
2,2018-01-02 01:24:05,1.0
3,2018-01-31 12:26:44,1.0
4,2018-03-01 23:51:21,1.0
...,...,...
406,2050-09-01 08:30:54,1.0
407,2050-09-30 16:31:48,1.0
408,2050-10-30 02:16:00,1.0
409,2050-11-28 14:09:48,1.0


In [229]:
# Timedelta + 15 days to create New_Moon Dataframe
new_moon = pd.DataFrame(full_moon['dtime'] + timedelta(days=14, hours=19, minutes=26))
new_moon['Phase'] = new_moon.loc['dtime',:] = -1
new_moon.drop(new_moon.index[-1], inplace=True)
new_moon['dtime'] = pd.to_datetime(new_moon['dtime'])

new_moon

,dtime,Phase
0,2017-11-18 23:48:55,-1.0
1,2017-12-18 10:12:59,-1.0
2,2018-01-16 20:50:05,-1.0
3,2018-02-15 07:52:44,-1.0
4,2018-03-16 19:17:21,-1.0
...,...,...
406,2050-09-16 03:56:54,-1.0
407,2050-10-15 11:57:48,-1.0
408,2050-11-13 21:42:00,-1.0
409,2050-12-13 09:35:48,-1.0


In [230]:
full_moon['dtime'] = pd.to_datetime(full_moon['dtime']).dt.date
new_moon['dtime'] = pd.to_datetime(new_moon['dtime']).dt.date

In [231]:
# Append Lunar Dataframes to create one table sorted by 'dtime'
phase_data = full_moon.append(new_moon)
phase_data.sort_values('dtime', inplace= True)
phase_data.reset_index(inplace=True)
phase_data.drop(['index'], axis = 1, inplace=True)
phase_data['dtime'] = pd.to_datetime(phase_data['dtime'])
# phase_data.set_index(['dtime'], inplace=True)
# phase_data

In [232]:
eclipse_df1 = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True)
eclipse_df1.rename(columns={'Date':'dtime'}, inplace=True)
eclipse_df1['Eclipse'] = 1
eclipse_df1['dtime'] = pd.to_datetime(eclipse_df1['dtime'])
# eclipse_df1

In [233]:
eclipse_df2 = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True)
eclipse_df2.rename(columns={'Date':'dtime'}, inplace=True)
eclipse_df2['Eclipse'] = 2
eclipse_df2['dtime'] = pd.to_datetime(eclipse_df2['dtime'])
eclipse_df2.drop(['Unnamed: 1'], axis = 1, inplace=True)
# eclipse_df2

In [234]:
eclipse_data = eclipse_df1.append(eclipse_df2)
eclipse_data.sort_values('dtime', inplace= True)
eclipse_data.reset_index(inplace=True)
eclipse_data.drop(['index'], axis = 1, inplace=True)
eclipse_data['dtime'] = pd.to_datetime(eclipse_data['dtime'])
eclipse_data.columns
eclipse_data.set_index(['dtime'], inplace=True)
eclipse_data.tail(365)

,Eclipse
dtime,
2020-11-30,1
2020-12-14,2
2021-05-26,1
2021-06-10,2
2021-11-19,1
...,...
2099-09-29,1
2100-02-24,1
2100-03-10,2


# Merge Dataframes on 'dtime'

In [235]:

moon_merge = btc_api.merge(phase_data, on='dtime', how='outer')
moon_merge.set_index(['dtime'], inplace=True)
moon_merge['Phase'] = moon_merge['Phase'].fillna(0)
moon_merge.dropna().head(40)

,Price,8SMA,200SMA,200EMA,Phase
dtime,,,,,
2018-01-15,13631.98,14128.39250,7054.67350,8292.518576,0.0
2018-01-16,11282.49,13666.68250,7098.68290,8316.017149,-1.0
2018-01-17,11162.70,13253.45750,7142.37335,8338.325261,0.0
2018-01-18,11175.52,12785.46125,7185.56865,8360.495582,0.0
2018-01-19,11521.76,12562.17375,7230.31510,8385.128304,0.0
2018-01-20,12783.94,12430.01750,7281.14820,8419.307773,0.0
2018-01-21,11549.93,12093.36875,7325.75855,8443.565406,0.0
2018-01-22,10814.52,11740.35500,7366.75995,8461.885943,0.0
2018-01-23,10858.23,11393.63625,7408.48170,8480.351849,0.0


In [236]:
eclipse_cat = moon_merge.merge(eclipse_data, on='dtime', how='outer')
eclipse_cat['Eclipse'] = eclipse_cat['Eclipse'].fillna(0)
# eclipse_cat['200SMA'] = eclipse_cat['200SMA'].fillna(0)
# eclipse_cat.set_index(['dtime'], inplace=True)
eclipse_cat.dropna()

,Price,8SMA,200SMA,200EMA,Phase,Eclipse
dtime,,,,,,
2018-01-15,13631.98,14128.39250,7054.67350,8292.518576,0.0,0.0
2018-01-16,11282.49,13666.68250,7098.68290,8316.017149,-1.0,0.0
2018-01-17,11162.70,13253.45750,7142.37335,8338.325261,0.0,0.0
2018-01-18,11175.52,12785.46125,7185.56865,8360.495582,0.0,0.0
2018-01-19,11521.76,12562.17375,7230.31510,8385.128304,0.0,0.0
...,...,...,...,...,...,...
2022-06-25,21474.19,20563.25875,38656.71040,38935.818669,0.0,0.0
2022-06-26,21031.85,20822.95875,38509.30445,38846.734174,0.0,0.0
2022-06-27,20718.16,20843.54375,38374.92600,38756.532162,0.0,0.0


### ****run data through an ML model to predict the price of BTC*

In [237]:
###
eclipse_cat['Actual Returns'] = eclipse_cat['Price'].pct_change().dropna()
eclipse_cat.head()

,Price,8SMA,200SMA,200EMA,Phase,Eclipse,Actual Returns
dtime,,,,,,,
2017-06-30,2480.61,NaN,NaN,2480.610000,0.0,0.0,NaN
2017-07-01,2424.61,NaN,NaN,2452.540175,0.0,0.0,-0.022575
2017-07-02,2536.46,NaN,NaN,2480.653083,0.0,0.0,0.046131
2017-07-03,2572.47,NaN,NaN,2503.779324,0.0,0.0,0.014197
2017-07-04,2617.32,NaN,NaN,2526.714538,0.0,0.0,0.017435


In [238]:
###
# initialize the new signal column 
eclipse_cat['Signal'] = 0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
eclipse_cat.loc[eclipse_cat['Actual Returns'] >= 0, 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
eclipse_cat.loc[eclipse_cat['Actual Returns'] < 0, 'Signal'] = -1

eclipse_cat.head()

,Price,8SMA,200SMA,200EMA,Phase,Eclipse,Actual Returns,Signal
dtime,,,,,,,,
2017-06-30,2480.61,NaN,NaN,2480.610000,0.0,0.0,NaN,0
2017-07-01,2424.61,NaN,NaN,2452.540175,0.0,0.0,-0.022575,-1
2017-07-02,2536.46,NaN,NaN,2480.653083,0.0,0.0,0.046131,1
2017-07-03,2572.47,NaN,NaN,2503.779324,0.0,0.0,0.014197,1
2017-07-04,2617.32,NaN,NaN,2526.714538,0.0,0.0,0.017435,1


In [239]:
####
test_df = eclipse_cat.shift().copy().dropna()

In [240]:
###
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = test_df[['8SMA', '200SMA']].shift().dropna() # have to shift 
X.head(5)
#X.shape

,8SMA,200SMA
dtime,,
2018-01-17,14128.39250,7054.67350
2018-01-18,13666.68250,7098.68290
2018-01-19,13253.45750,7142.37335
2018-01-20,12785.46125,7185.56865
2018-01-21,12562.17375,7230.31510


In [241]:
###
# Create the target set selecting the Signal column and assiging it to y
y = test_df['Signal']
y.value_counts()

 1.0    853
-1.0    774
Name: Signal, dtype: int64

In [242]:
###
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2018-01-17 00:00:00


In [243]:
###
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

2018-04-17 00:00:00


In [244]:
###
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,8SMA,200SMA
dtime,,
2018-01-17,14128.39250,7054.67350
2018-01-18,13666.68250,7098.68290
2018-01-19,13253.45750,7142.37335
2018-01-20,12785.46125,7185.56865
2018-01-21,12562.17375,7230.31510


In [245]:
###
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(day=1):]
y_test = y.loc[training_end+DateOffset(day=1):]

# Review the X_test DataFrame
X_train.head()

,8SMA,200SMA
dtime,,
2018-01-17,14128.39250,7054.67350
2018-01-18,13666.68250,7098.68290
2018-01-19,13253.45750,7142.37335
2018-01-20,12785.46125,7185.56865
2018-01-21,12562.17375,7230.31510


In [246]:
###
y_test.head()

dtime
2018-04-01    1.0
2018-04-02   -1.0
2018-04-03    1.0
2018-04-04    1.0
2018-04-05   -1.0
Name: Signal, dtype: float64

In [247]:
###
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [248]:
###
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC() 
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred


array([ 1.,  1.,  1., ..., -1., -1., -1.])

In [249]:
###
# display length of svm_pred array 
len(svm_pred)

1552

In [250]:
###
y_test.describe()

count    1552.000000
mean        0.054124
std         0.998856
min        -1.000000
25%        -1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Signal, dtype: float64

In [251]:
###
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

        -1.0       0.47      0.73      0.58       734
         1.0       0.53      0.27      0.36       818

    accuracy                           0.49      1552
   macro avg       0.50      0.50      0.47      1552
weighted avg       0.50      0.49      0.46      1552



In [252]:
###
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index) 

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = eclipse_cat['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = predictions_df['Actual Returns']*predictions_df['Predicted']

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail()) 

,Predicted,Actual Returns,Strategy Returns
dtime,,,
2018-04-01,1.0,-0.015543,-0.015543
2018-04-02,1.0,0.034928,0.034928
2018-04-03,1.0,0.050844,0.050844
2018-04-04,1.0,-0.083236,-0.083236
2018-04-05,1.0,-0.003675,-0.003675


,Predicted,Actual Returns,Strategy Returns
dtime,,,
2022-06-26,-1.0,-0.020599,0.020599
2022-06-27,-1.0,-0.014915,0.014915
2022-06-28,-1.0,-0.022502,0.022502
2022-06-29,-1.0,0.001739,-0.001739
2022-07-13,-1.0,0.000000,-0.000000


In [253]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [dtime]   (value)

In [254]:

def Tru_Moo(x, y): # Tru_Moo = True tells us that if you bought at full moon price and sold at new moon price you were profitable 
    if x - y > 0:
        return True
    return False

moon_data['Tru_Moo'] = moon_data.apply(lambda row: Tru_Moo(row['New Moon Price'], row['Full Moon Price']), axis=1)
moon_data['PCT Change'] = (moon_data['New Moon Price'] - moon_data['Full Moon Price']) / moon_data['New Moon Price']

#for ind in (moon_data.index):
#print(moon_data.iloc[:,2])
#print(moon_data.iloc[1:,3]- moon_data.iloc[:,2])


moon_data

,New_Moon,New Moon Price,Full_Moon,Full Moon Price,Tru_Moo,PCT Change
0,2017-11-19,8042.64,2017-11-04,7363.80,True,0.084405
1,2017-12-18,18972.32,2017-12-03,11246.21,True,0.407231
2,2018-01-17,11162.70,2018-01-02,14754.13,False,-0.321735
3,2018-02-15,10033.75,2018-01-31,10226.86,False,-0.019246
4,2018-03-17,7883.45,2018-03-02,11043.12,False,-0.400798
5,2018-04-15,8376.73,2018-03-31,6943.77,True,0.171064
6,2018-05-15,8480.16,2018-04-30,9248.45,False,-0.090599
7,2018-06-13,6310.43,2018-05-29,7474.75,False,-0.184507
8,2018-07-13,6229.83,2018-06-28,5871.28,True,0.057554
9,2018-08-11,6242.54,2018-07-27,8182.89,False,-0.310827


In [255]:
# Create a New Temporary Data Frame to Finalize the Moon Price Behavior Data

frame = {'Full Moon': moon_data.iloc[:,0], 'New Moon': moon_data.iloc[:,1], 'Full Moon Price': moon_data.iloc[:,2], 'New Moon Price': moon_data.iloc[:,3]} #'New Moon Price': moon_data.iloc[1:,3]}
tmpdf = pd.DataFrame(frame)
tmpdf['Difference']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'] #.shift(1)
tmpdf['Percentage Difference'] = ((tmpdf['Full Moon Price'] - tmpdf['New Moon Price']) / tmpdf['New Moon Price'])*100
tmpdf['Trade Profit']= tmpdf['Difference'].cumsum()
tmpdf['Buy and Hold']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'][0]

def returnBoolFM(x):
    if x < 0:
        return True
    return False

def returnBoolNM(x):
    if x > 0:
        return True
    return False

tmpdf['Did We Profit this Month'] = tmpdf.apply(lambda row: returnBoolNM(row['Difference']), axis = 1)


tmpdf

TypeError: cannot subtract DatetimeArray from ndarray

In [ ]:
#plot Difference column vs the buy and hold column
buy_hold_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Buy and Hold',
    xlabel = 'Date',
    ylabel = 'Buy and Hold',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'red',
    legend = True
)

moon_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Trade Profit',
    xlabel = 'Date',
    ylabel = 'Difference',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'blue',
    legend = True
    
)

moon_glyph*buy_hold_glyph

In [ ]:
# Define a function to count the Trues in the 'Did We Profit this Month' column
def count_tr(x):
    return x.sum()
number_of_Trues = (count_tr(tmpdf['Did We Profit this Month']))

#Count the number of rows in the Did We Profit this Month column

number_of_Rows = tmpdf['Did We Profit this Month'].count()

# The PERCENTAGE of times you would profit if you bought on Full Moon and sold on New Moon each month

print(f'{(number_of_Trues/number_of_Rows)*100:.2f}% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!')

In [ ]:
#7. create column for the all time high and low
# moon_data['All Time High'] = Price.max()
# moon_data


In [ ]:
fear_data = json.dumps(FearGreedUrl["data"][0], indent=4)
fear_json = json.loads(fear_data)
fear_json


In [ ]:
fear_df = pd.DataFrame(data=fear_json, columns=['value', 'value_classification', 'timestamp'])